# Projeto 2 - Ciência dos Dados

Nome: Thomas Bekhor

Nome: Luca Machado

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [29]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [30]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
from IPython.display import display
pd.options.display.max_rows = 13

___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***

In [31]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [32]:
#Produto escolhido:
produto = "Uber"

#Quantidade mínima de mensagens capturadas:
n = 590
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [33]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
msgs = list(dict.fromkeys(msgs))
shuffle(msgs)


Salvando os dados em uma planilha Excel:

In [34]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)
    
    #fecha o arquivo
    writer.save()
    

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [35]:
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

In [36]:
#lendo cada planilha separadamente do excel
twitter_treinamento = pd.read_excel(io="./Uber.xlsx",sheet_name="Treinamento")
twitter_teste = pd.read_excel(io="./Uber.xlsx",sheet_name="Teste")

#aplicando a função cleanup no "Teste" e tirando as maiúsculas
twitter_teste["Teste s/ pont."] = twitter_teste["Teste"].apply(cleanup)
twitter_teste=twitter_teste.drop(columns="Teste")
twitter_teste=twitter_teste[["Teste s/ pont.", "Relevância"]]
twitter_teste["Teste s/ pont."]=twitter_teste["Teste s/ pont."].str.lower()

#aplicando a função cleanup no "Treinamento"  e tirando as maiúsculas
twitter_treinamento["Treinamento s/ pont."] = twitter_treinamento["Treinamento"].apply(cleanup)
twitter_treinamento=twitter_treinamento.drop(columns="Treinamento")
twitter_treinamento=twitter_treinamento[["Treinamento s/ pont.", "Relevância"]]
twitter_treinamento["Treinamento s/ pont."]=twitter_treinamento["Treinamento s/ pont."].str.lower()



In [37]:
twitter_treinamento

,Treinamento s/ pont.,Relevância
0,@beatriz_seca uber não sou demasiado chique p...,0
1,parece que o uber que eu vou pegar é bonito ht...,0
2,@lucascollar eu pago o uber,0
3,uber investe us 200 milhões em serviço que nã...,0
4,moss alguel que usa uber eats me ajuda eu imploro,0
5,@piresejunior @uber_brasil não caia nesse papo...,1
...,...,...
294,ônibus= 4 reais\n\nuber= 12 reais\n\nminha mãe...,1
295,rt @luchotessadri sos pelotudo o pensas que u...,0
296,indo de uber pra casa 😎 acabo a mamata porra,0


In [38]:
#separando as palaras numa lista só
series_treinamento=[]
for i in twitter_treinamento["Treinamento s/ pont."]:
    for n in i.split():
        series_treinamento.append(n)
series_treinamento

#separando as palavras para os relevantes
series_treinamento_rel=[]
for i in twitter_treinamento[twitter_treinamento["Relevância"]==1]["Treinamento s/ pont."]:
    for n in i.split():
        series_treinamento_rel.append(n)

#separando as palavras para os não relevantes
series_treinamento_nrel=[]
for i in twitter_treinamento[twitter_treinamento["Relevância"]==0]["Treinamento s/ pont."]:
    for n in i.split():
        series_treinamento_nrel.append(n)
print(len(series_treinamento))

5664


In [47]:
#frequencia relativa nao relevantes
frame_rel=pd.Series(series_treinamento_rel)
frame_rel_relativo=frame_rel.value_counts(True)

#frequencia relativa relevantes
frame_nrel=pd.Series(series_treinamento_nrel)
frame_nrel_relativo=frame_nrel.value_counts(True)

#frequencia relativa total
frame_total=pd.Series(series_treinamento)
frame_total_relativo=frame_total.value_counts(True)

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [50]:

for i in twitter_teste[twitter_teste["Relevância"]==0]["Teste s/ pont."]:
    for n in i.split():
        for f in frame_nrel_relativo:
            if n==f:
                twitter_teste["Verificação"]=1
            else:
                twitter_teste["Verificação"]=0
    
    
for i in twitter_teste[twitter_teste["Relevância"]==1]["Teste s/ pont."]:
    for n in i.split():
        for f in frame_rel_relativo:
            if n==f:
                twitter_teste["Verificação"]=1
            else:
                twitter_teste["Verificação"]=0


In [49]:
twitter_teste

,Teste s/ pont.,Relevância,Verificação
0,tava tão doida p ir embora daquele lugar q ent...,0,0
1,inclusive quando vi o da campus party quase fa...,0,0
2,status cantando sertanejo com o uber,1,0
3,uber bom é uber que bate o carro no cara de trás,1,0
4,e eu q pedi um uber aí tava nome de um homem l...,1,0
5,trêm//metrô 4 30\nuber 16 90\n\neu quase o ...,1,0
...,...,...,...
194,decidi pegar uber,0,0
195,uber sem ar condicionado é sacanagi,1,0
196,tô muito mau acostumada c uber esse fim de s...,0,0


___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**